### Code Hist.

 - CODE : Clustering - 라벨에 따른 사용량 분리 및 저장  
 - DESC : 군집화 - 단지 전체의 에너지 사용량을 Cluster별로 분류하여 CSV 저장  
 - DATE  
    &ensp; 2024-02-22 Created  
    &ensp; 2024-02-25 Updated : 날씨 데이터 Merge 부분을 분리 및 제거 (intergration_Weather)  
    &ensp; 2024-08-05 Updated : Error Fix  

# 01. Code

## 01-01. Init

### 01-01-01. Init_Module Import

In [13]:
#region Basic_Import
## Basic
import os, sys, warnings
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.path.dirname(os.path.abspath('./__file__'))
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('./__file__'))))
warnings.filterwarnings('ignore')

import numpy as np, pandas as pd
from pandas import DataFrame, Series
pd.options.display.float_format = '{:.10f}'.format

import math, random

## Datetime
import time, datetime as dt
from datetime import datetime, date, timedelta

## glob
import glob, requests, json
from glob import glob

## 시각화
import matplotlib.pyplot as plt, seaborn as sns
# %matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]

from scipy import stats

## Split, 정규화
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# K-Means 알고리즘
from sklearn.cluster import KMeans, MiniBatchKMeans

# Clustering 알고리즘의 성능 평가 측도
from sklearn import metrics
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score, adjusted_rand_score, silhouette_score, rand_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.metrics.cluster import contingency_matrix

## For Web
import urllib
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
#endregion Basic_Import

In [14]:
## Import_DL
str_tar = "tf"
## For Torch
if str_tar == "torch":
    import torch, torch.nn as nn
    from torch.nn.utils import weight_norm
    print("Torch Imported")
## For TF
elif str_tar == "tf":
    import tensorflow as tf, tensorflow_addons as tfa
    from keras.callbacks import EarlyStopping, ModelCheckpoint
    from keras.models import Sequential, load_model
    from keras_flops import get_flops
    print("Tensorflow Imported")
else:
    print("Error : Cannot be used except for Keywords")
    print(" : torch / tf")

Tensorflow Imported


In [15]:
## Import_Local
from Src_Dev_Common import Data_Datetime as com_date, KMA_Weather as com_KMA, KECO_AirKor as com_KECO, KASI_Holiday as com_Holi, KIER_Usage_M02 as com_KIER_M02

### 01-01-02. Config (Directory, Params)

In [16]:
## Init_config
SEED = 42

np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)
os.environ["PYTHONHASHSEED"], os.environ['TF_DETERMINISTIC_OPS'] = str(SEED), "1"

In [17]:
## Define Todate str
str_now_ymd = pd.datetime.now().date()
str_now_y, str_now_m, str_now_d = pd.datetime.now().year, pd.datetime.now().month, pd.datetime.now().day
str_now_hr, str_now_min = pd.datetime.now().hour, pd.datetime.now().minute

print(pd.datetime.now())
print(str(str_now_y) + " / " + str(str_now_m)  + " / " + str(str_now_d))
print(str(str_now_hr) + " : " + str(str_now_min))

2024-08-05 15:55:36.911345
2024 / 8 / 5
15 : 55


In [18]:
## Dict_Domain
## {0:"ELEC", 1:"HEAT", 2:"WATER", 3:"HOT_HEAT", 4:"HOT_FLOW", 99:"GAS"}
## {'10MIN', '1H', '1D', '1W', '1M'}
int_domain, int_interval = 0, 0

## Domain, ACCU/INST Column
str_domain, str_col_accu, str_col_inst = com_KIER_M02.create_domain_str(int_domain)
## Directory Root
str_dirData, str_dir_raw, str_dir_cleansed, str_dirName_bld, str_dirName_h = com_KIER_M02.create_dir_str(str_domain)
## Interval, Target File
str_interval, str_fileRaw, str_fileRaw_hList, str_file = com_KIER_M02.create_file_str(str_domain, int_interval)

print(str(os.listdir(str_dirData)) + "\n")
print(os.listdir(str_dirName_h))

0 : ELEC
str_fileRaw : KIER_RAW_ELEC_2024-06-07.csv
str_fileRaw_hList : KIER_RAW_ELEC_2024-06-07.csv
str_file : KIER_ELEC_INST_03-01_10MIN.csv
['.ipynb_checkpoints', 'BS_CONFIGURATION_202309251452.csv', 'DATE_1M_2023-10-20.csv', 'KIER 전처리 현황_2024-06-25.xlsx', 'KIER_0_Raw', 'KIER_1_Cleansed', 'KIER_2_BLD', 'KIER_3_H_ELEC', 'KIER_3_H_GAS', 'KIER_3_H_HEAT', 'KIER_3_H_HOT_FLOW', 'KIER_3_H_HOT_HEAT', 'KIER_3_H_WATER', 'KIER_ASOS_WEATHER_DAILY_202309251521.csv', 'KIER_ASOS_WEATHER_HOUR_202309251521.csv', 'KIER_DATA_OLD', 'KIER_ETC', 'KIER_hList_Comparison_2024-06-26.xlsx', 'KIER_List_Table_Column_2023-09-25.xlsx', 'KIER_Query_2023-09-25.txt', 'KMA_ASOS_119_2010_2023_1st_to CSV.csv', '[IITP] 데이터 테이블 정리 (공유 원본).docx']

['KIER_ELEC_561-1-1_ACCU_01_Raw.csv', 'KIER_ELEC_561-1-2_ACCU_01_Raw.csv', 'KIER_ELEC_561-1-3_ACCU_01_Raw.csv', 'KIER_ELEC_561-1-4_ACCU_01_Raw.csv', 'KIER_ELEC_561-10-1_ACCU_01_Raw.csv', 'KIER_ELEC_561-10-2_ACCU_01_Raw.csv', 'KIER_ELEC_561-10-3_ACCU_01_Raw.csv', 'KIER_ELEC_561-1

## 01-02. Data Load (df_raw)

### 01-02-01. KIER Energy Usage (Intergrated)

In [19]:
str_file = 'KIER_' + str_domain + '_Input_Comb_' + str_interval + '.csv'
df_kier_raw = pd.read_csv(str_dirName_h + str_file, index_col = 0)
df_kier_raw

,METER_DATE,YEAR,MONTH,DAY,HOUR,MINUTE,temp_outdoor,temp_dew_point,temp_ground,humidity,...,ELEC_INST_EFF_563-22-5,ELEC_INST_EFF_563-22-6,ELEC_INST_EFF_563-23-1,ELEC_INST_EFF_563-23-2,ELEC_INST_EFF_563-23-3,ELEC_INST_EFF_563-23-4,ELEC_INST_EFF_563-24-1,ELEC_INST_EFF_563-24-2,MEAN_OF_INST,SUM_OF_INST
0,2022-07-17 23:20:00,2022,7,17,23,20,NaN,NaN,NaN,NaN,...,0.0300000000,0.0400000000,0.0500000000,0.0800000000,0.0600000000,0.0600000000,0.0300000000,0.1000000000,0.0810000000,27.5400000000
1,2022-07-17 23:30:00,2022,7,17,23,30,NaN,NaN,NaN,NaN,...,0.0200000000,0.0400000000,0.0600000000,0.0600000000,0.0500000000,0.0772916667,0.0200000000,0.0900000000,0.0772916667,25.9700000000
2,2022-07-17 23:40:00,2022,7,17,23,40,NaN,NaN,NaN,NaN,...,0.0400000000,0.0300000000,0.0500000000,0.0600000000,0.0500000000,0.0759064327,0.0300000000,0.0900000000,0.0759064327,25.9600000000
3,2022-07-17 23:50:00,2022,7,17,23,50,NaN,NaN,NaN,NaN,...,0.0400000000,0.0400000000,0.0400000000,0.0600000000,0.0400000000,0.0500000000,0.0300000000,0.0800000000,0.0738053097,25.0200000000
4,2022-07-18 00:00:00,2022,7,18,0,0,24.0000000000,21.3000000000,23.4000000000,85.0000000000,...,0.0400000000,0.0400000000,0.0300000000,0.0600000000,0.0600000000,0.0300000000,0.0200000000,0.0600000000,0.0706528190,23.8100000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99165,2024-06-05 14:50:00,2024,6,5,14,50,26.6333333333,14.3500000000,41.5000000000,47.0000000000,...,0.0400000000,0.0511049724,0.0511049724,0.0511049724,0.0511049724,0.0511049724,0.0511049724,0.0511049724,0.0511049724,9.2500000000
99166,2024-06-05 15:00:00,2024,6,5,15,0,26.3000000000,14.4000000000,41.1000000000,48.0000000000,...,0.0800000000,0.0477714286,0.0477714286,0.0477714286,0.0477714286,0.0477714286,0.0477714286,0.0477714286,0.0477714286,8.3600000000
99167,2024-06-05 15:10:00,2024,6,5,15,10,26.3000000000,14.4000000000,41.1000000000,48.0000000000,...,0.0500000000,0.0453977273,0.0453977273,0.0453977273,0.0453977273,0.0453977273,0.0453977273,0.0453977273,0.0453977273,7.9900000000
99168,2024-06-05 15:20:00,2024,6,5,15,20,26.3000000000,14.4000000000,41.1000000000,48.0000000000,...,0.0900000000,0.0515517241,0.0515517241,0.0515517241,0.0515517241,0.0515517241,0.0515517241,0.0515517241,0.0515517241,5.9800000000


In [20]:
df_kier_raw.columns[25:]

Index(['DAY.1', 'HOUR.1', 'MINUTE.1', 'ELEC_INST_EFF_561-1-1',
       'ELEC_INST_EFF_561-1-2', 'ELEC_INST_EFF_561-1-3',
       'ELEC_INST_EFF_561-1-4', 'ELEC_INST_EFF_561-2-1',
       'ELEC_INST_EFF_561-2-2', 'ELEC_INST_EFF_561-2-3',
       ...
       'ELEC_INST_EFF_563-22-5', 'ELEC_INST_EFF_563-22-6',
       'ELEC_INST_EFF_563-23-1', 'ELEC_INST_EFF_563-23-2',
       'ELEC_INST_EFF_563-23-3', 'ELEC_INST_EFF_563-23-4',
       'ELEC_INST_EFF_563-24-1', 'ELEC_INST_EFF_563-24-2', 'MEAN_OF_INST',
       'SUM_OF_INST'],
      dtype='object', length=353)

In [28]:
## Column List
list_col_date = df_kier_raw.columns[:6].to_list() ## Date Data
list_col_weather = df_kier_raw.columns[7:23].to_list() ## Weather Data
df_kier_raw_weather = df_kier_raw[list_col_weather]
df_kier_raw_weather.insert(0, 'METER_DATE', df_kier_raw['METER_DATE'])

## Usage Data
list_kier_h_all = df_kier_raw.columns[28:].to_list()

print(list_col_date)
print(list_col_weather)
print(list_kier_h_all)

['METER_DATE', 'YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE']
['temp_dew_point', 'temp_ground', 'humidity', 'rainfall', 'snowfall', 'snowfall_3hr', 'wind_speed', 'wind_direction', 'pressure_vapor', 'pressure_area', 'pressure_sea', 'sunshine', 'solar_radiation', 'cloud_total', 'cloud_midlow', 'visual_range']
['ELEC_INST_EFF_561-1-1', 'ELEC_INST_EFF_561-1-2', 'ELEC_INST_EFF_561-1-3', 'ELEC_INST_EFF_561-1-4', 'ELEC_INST_EFF_561-2-1', 'ELEC_INST_EFF_561-2-2', 'ELEC_INST_EFF_561-2-3', 'ELEC_INST_EFF_561-2-4', 'ELEC_INST_EFF_561-3-1', 'ELEC_INST_EFF_561-3-2', 'ELEC_INST_EFF_561-3-3', 'ELEC_INST_EFF_561-3-4', 'ELEC_INST_EFF_561-4-1', 'ELEC_INST_EFF_561-4-2', 'ELEC_INST_EFF_561-4-3', 'ELEC_INST_EFF_561-4-4', 'ELEC_INST_EFF_561-5-1', 'ELEC_INST_EFF_561-5-2', 'ELEC_INST_EFF_561-5-3', 'ELEC_INST_EFF_561-5-4', 'ELEC_INST_EFF_561-6-1', 'ELEC_INST_EFF_561-6-2', 'ELEC_INST_EFF_561-6-3', 'ELEC_INST_EFF_561-6-4', 'ELEC_INST_EFF_561-7-1', 'ELEC_INST_EFF_561-7-2', 'ELEC_INST_EFF_561-7-3', 'ELEC_INST_EFF_561-7

In [22]:
## 직전 시간의 에너지 사용 Column
df_kier_raw_prev = df_kier_raw[list_col_date + list_kier_h_all]
df_kier_raw_prev['METER_DATE'] = df_kier_raw_prev['METER_DATE'].shift(-1)
df_kier_raw_prev = df_kier_raw_prev[:-1]

In [24]:
## Label별 호실 List Load (From "M02-02_Clustering_02_KMeans_Find K.ipynb")
K = 2
str_file_clustering = 'KIER_' + str(str_domain) + '_Labeled_' + str_interval + '_K' + str(K) + '.csv'

df_kier_h_cluster = pd.read_csv(str_dirName_h + str_file_clustering, index_col = 0).rename(columns = {'index' : 'h_index'})[['h_index', 'target_' + str_domain]]
list_kier_h_c0 = df_kier_h_cluster[df_kier_h_cluster['target_' + str_domain] == 0]['h_index']
print(len(list_kier_h_c0))
list_kier_h_c1 = df_kier_h_cluster[df_kier_h_cluster['target_' + str_domain] == 1]['h_index']
print(len(list_kier_h_c1))
list_kier_h_c2 = df_kier_h_cluster[df_kier_h_cluster['target_' + str_domain] == 2]['h_index']
print(len(list_kier_h_c2))

118
230
0


In [29]:
## 전체 사용량 합계
df_kier_h_all = df_kier_raw_weather.copy()
df_kier_h_tmp = df_kier_raw[list_kier_h_all]
df_kier_h_all['ELEC_INST_SUM'] = df_kier_h_tmp.sum(axis = 1)
df_kier_h_tmp['METER_DATE'] = df_kier_raw_weather['METER_DATE']
df_kier_h_all = pd.merge(df_kier_h_all, df_kier_raw_prev, how = 'left', on = 'METER_DATE').dropna()

## Cluster별 사용량 합계
## ■ C00
df_kier_h_c0 = df_kier_raw_weather.copy()
df_kier_h_tmp = df_kier_raw[list_kier_h_c0]
df_kier_h_c0['ELEC_INST_SUM_C0'] = df_kier_h_tmp.sum(axis = 1)
df_kier_h_tmp['METER_DATE'] = df_kier_raw_weather['METER_DATE']
df_kier_h_c0['ELEC_INST_SUM_C0'] = df_kier_h_tmp.sum(axis = 1)
df_kier_h_c0 = pd.merge(df_kier_h_c0, df_kier_raw_prev, how = 'left', on = 'METER_DATE').dropna()

# ## ■ C01
df_kier_h_c1 = df_kier_raw_weather.copy()
df_kier_h_tmp = df_kier_raw[list_kier_h_c1]
df_kier_h_c1['ELEC_INST_SUM_C1'] = df_kier_h_tmp.sum(axis = 1)
df_kier_h_tmp['METER_DATE'] = df_kier_raw_weather['METER_DATE']
df_kier_h_c1['ELEC_INST_SUM_C1'] = df_kier_h_tmp.sum(axis = 1)
df_kier_h_c1 = pd.merge(df_kier_h_c1, df_kier_raw_prev, how = 'left', on = 'METER_DATE').dropna()

# ## ■ C02
df_kier_h_c2 = df_kier_raw_weather.copy()
df_kier_h_tmp = df_kier_raw[list_kier_h_c2]
df_kier_h_c2['ELEC_INST_SUM_C2'] = df_kier_h_tmp.sum(axis = 1)
df_kier_h_tmp['METER_DATE'] = df_kier_raw_weather['METER_DATE']
df_kier_h_c2['ELEC_INST_SUM_C2'] = df_kier_h_tmp.sum(axis = 1)
df_kier_h_c2 = pd.merge(df_kier_h_c2, df_kier_raw_prev, how = 'left', on = 'METER_DATE').dropna()

df_kier_h_c2

,METER_DATE,temp_dew_point,temp_ground,humidity,rainfall,snowfall,snowfall_3hr,wind_speed,wind_direction,pressure_vapor,...,ELEC_INST_EFF_563-22-5,ELEC_INST_EFF_563-22-6,ELEC_INST_EFF_563-23-1,ELEC_INST_EFF_563-23-2,ELEC_INST_EFF_563-23-3,ELEC_INST_EFF_563-23-4,ELEC_INST_EFF_563-24-1,ELEC_INST_EFF_563-24-2,MEAN_OF_INST,SUM_OF_INST
4,2022-07-18 00:00:00,21.3000000000,23.4000000000,85.0000000000,0.0000000000,0.0000000000,0.0000000000,0.5000000000,250.0000000000,25.3000000000,...,0.0400000000,0.0400000000,0.0400000000,0.0600000000,0.0400000000,0.0500000000,0.0300000000,0.0800000000,0.0738053097,25.0200000000
5,2022-07-18 00:10:00,21.3666666667,23.3333333333,85.6666666667,0.0000000000,0.0000000000,0.0000000000,0.5000000000,226.6666666667,25.4166666667,...,0.0400000000,0.0400000000,0.0300000000,0.0600000000,0.0600000000,0.0300000000,0.0200000000,0.0600000000,0.0706528190,23.8100000000
6,2022-07-18 00:20:00,21.4333333333,23.2666666667,86.3333333333,0.0000000000,0.0000000000,0.0000000000,0.5000000000,203.3333333333,25.5333333333,...,0.0400000000,0.0400000000,0.0300000000,0.0700000000,0.0400000000,0.0200000000,0.0300000000,0.0800000000,0.0687500000,23.6500000000
7,2022-07-18 00:30:00,21.5000000000,23.2000000000,87.0000000000,0.0000000000,0.0000000000,0.0000000000,0.5000000000,180.0000000000,25.6500000000,...,0.0200000000,0.0300000000,0.0400000000,0.0400000000,0.0300000000,0.0400000000,0.0200000000,0.0600000000,0.0646376812,22.3000000000
8,2022-07-18 00:40:00,21.5666666667,23.1333333333,87.6666666667,0.0000000000,0.0000000000,0.0000000000,0.5000000000,156.6666666667,25.7666666667,...,0.0200000000,0.0400000000,0.0400000000,0.0600000000,0.0500000000,0.0300000000,0.0400000000,0.0700000000,0.0615406977,21.1700000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99165,2024-06-05 14:50:00,14.3500000000,41.5000000000,47.0000000000,0.0000000000,0.0000000000,0.0000000000,3.0000000000,278.3333333333,16.3500000000,...,0.0200000000,0.0503743316,0.0300000000,0.0503743316,0.0503743316,0.0503743316,0.0503743316,0.0503743316,0.0503743316,9.4200000000
99166,2024-06-05 15:00:00,14.4000000000,41.1000000000,48.0000000000,0.0000000000,0.0000000000,0.0000000000,3.1000000000,270.0000000000,16.4000000000,...,0.0400000000,0.0511049724,0.0511049724,0.0511049724,0.0511049724,0.0511049724,0.0511049724,0.0511049724,0.0511049724,9.2500000000
99167,2024-06-05 15:10:00,14.4000000000,41.1000000000,48.0000000000,0.0000000000,0.0000000000,0.0000000000,3.1000000000,270.0000000000,16.4000000000,...,0.0800000000,0.0477714286,0.0477714286,0.0477714286,0.0477714286,0.0477714286,0.0477714286,0.0477714286,0.0477714286,8.3600000000
99168,2024-06-05 15:20:00,14.4000000000,41.1000000000,48.0000000000,0.0000000000,0.0000000000,0.0000000000,3.1000000000,270.0000000000,16.4000000000,...,0.0500000000,0.0453977273,0.0453977273,0.0453977273,0.0453977273,0.0453977273,0.0453977273,0.0453977273,0.0453977273,7.9900000000


In [ ]:
# ## 전체 사용량 합계
# df_kier_h_all = df_kier_raw[list_kier_h_all]
# df_kier_h_all['ELEC_INST_SUM'] = df_kier_h_all.sum(axis = 1)
# df_kier_h_all.insert(0, 'METER_DATE', df_kier_raw['METER_DATE'])
# df_kier_h_all = pd.merge(df_kier_raw_weather, df_kier_h_all
#                          , how = 'left', on = ['METER_DATE'])
# list_kier_h_all = ['METER_DATE'] + df_kier_raw_weather + ['ELEC_INST_SUM']
# df_kier_h_all = df_kier_h_all[list_kier_h_all]

# ## Cluster별 사용량 합계
# ## ■ C00
# df_kier_h_c0 = df_kier_raw[list_kier_h_c0]
# df_kier_h_c0['ELEC_INST_SUM_C0'] = df_kier_h_c0.sum(axis = 1)
# df_kier_h_c0.insert(0, 'METER_DATE', df_kier_raw['METER_DATE'])
# ## 합계행만 남기기 (필요시 세대별 사용량 복원 가능)
# df_kier_h_c0 = df_kier_h_c0[['METER_DATE', 'ELEC_INST_SUM_C0']]

# ## ■ C01
# df_kier_h_c1 = df_kier_raw[list_kier_h_c1]
# df_kier_h_c1['ELEC_INST_SUM_C1'] = df_kier_h_c1.sum(axis = 1)
# df_kier_h_c1.insert(0, 'METER_DATE', df_kier_raw['METER_DATE'])
# ## 합계행만 남기기 (필요시 세대별 사용량 복원 가능)
# df_kier_h_c1 = df_kier_h_c1[['METER_DATE', 'ELEC_INST_SUM_C1']]

# ## ■ C02
# df_kier_h_c2 = df_kier_raw[list_kier_h_c2]
# df_kier_h_c2['ELEC_INST_SUM_C2'] = df_kier_h_c2.sum(axis = 1)
# df_kier_h_c2.insert(0, 'METER_DATE', df_kier_raw['METER_DATE'])
# ## 합계행만 남기기 (필요시 세대별 사용량 복원 가능)
# df_kier_h_c2 = df_kier_h_c2[['METER_DATE', 'ELEC_INST_SUM_C2']]

In [30]:
df_kier_h_all

,METER_DATE,temp_dew_point,temp_ground,humidity,rainfall,snowfall,snowfall_3hr,wind_speed,wind_direction,pressure_vapor,...,ELEC_INST_EFF_563-22-5,ELEC_INST_EFF_563-22-6,ELEC_INST_EFF_563-23-1,ELEC_INST_EFF_563-23-2,ELEC_INST_EFF_563-23-3,ELEC_INST_EFF_563-23-4,ELEC_INST_EFF_563-24-1,ELEC_INST_EFF_563-24-2,MEAN_OF_INST,SUM_OF_INST
4,2022-07-18 00:00:00,21.3000000000,23.4000000000,85.0000000000,0.0000000000,0.0000000000,0.0000000000,0.5000000000,250.0000000000,25.3000000000,...,0.0400000000,0.0400000000,0.0400000000,0.0600000000,0.0400000000,0.0500000000,0.0300000000,0.0800000000,0.0738053097,25.0200000000
5,2022-07-18 00:10:00,21.3666666667,23.3333333333,85.6666666667,0.0000000000,0.0000000000,0.0000000000,0.5000000000,226.6666666667,25.4166666667,...,0.0400000000,0.0400000000,0.0300000000,0.0600000000,0.0600000000,0.0300000000,0.0200000000,0.0600000000,0.0706528190,23.8100000000
6,2022-07-18 00:20:00,21.4333333333,23.2666666667,86.3333333333,0.0000000000,0.0000000000,0.0000000000,0.5000000000,203.3333333333,25.5333333333,...,0.0400000000,0.0400000000,0.0300000000,0.0700000000,0.0400000000,0.0200000000,0.0300000000,0.0800000000,0.0687500000,23.6500000000
7,2022-07-18 00:30:00,21.5000000000,23.2000000000,87.0000000000,0.0000000000,0.0000000000,0.0000000000,0.5000000000,180.0000000000,25.6500000000,...,0.0200000000,0.0300000000,0.0400000000,0.0400000000,0.0300000000,0.0400000000,0.0200000000,0.0600000000,0.0646376812,22.3000000000
8,2022-07-18 00:40:00,21.5666666667,23.1333333333,87.6666666667,0.0000000000,0.0000000000,0.0000000000,0.5000000000,156.6666666667,25.7666666667,...,0.0200000000,0.0400000000,0.0400000000,0.0600000000,0.0500000000,0.0300000000,0.0400000000,0.0700000000,0.0615406977,21.1700000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99165,2024-06-05 14:50:00,14.3500000000,41.5000000000,47.0000000000,0.0000000000,0.0000000000,0.0000000000,3.0000000000,278.3333333333,16.3500000000,...,0.0200000000,0.0503743316,0.0300000000,0.0503743316,0.0503743316,0.0503743316,0.0503743316,0.0503743316,0.0503743316,9.4200000000
99166,2024-06-05 15:00:00,14.4000000000,41.1000000000,48.0000000000,0.0000000000,0.0000000000,0.0000000000,3.1000000000,270.0000000000,16.4000000000,...,0.0400000000,0.0511049724,0.0511049724,0.0511049724,0.0511049724,0.0511049724,0.0511049724,0.0511049724,0.0511049724,9.2500000000
99167,2024-06-05 15:10:00,14.4000000000,41.1000000000,48.0000000000,0.0000000000,0.0000000000,0.0000000000,3.1000000000,270.0000000000,16.4000000000,...,0.0800000000,0.0477714286,0.0477714286,0.0477714286,0.0477714286,0.0477714286,0.0477714286,0.0477714286,0.0477714286,8.3600000000
99168,2024-06-05 15:20:00,14.4000000000,41.1000000000,48.0000000000,0.0000000000,0.0000000000,0.0000000000,3.1000000000,270.0000000000,16.4000000000,...,0.0500000000,0.0453977273,0.0453977273,0.0453977273,0.0453977273,0.0453977273,0.0453977273,0.0453977273,0.0453977273,7.9900000000


## 01-03. Data Integeration

In [31]:
str_file = 'KIER_' + str_domain + '_INST_Weather_ALL_' + str_interval + '.csv'
df_kier_h_all.to_csv(str_dirName_h + str_file)
print(df_kier_h_all.info())
df_kier_h_all

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99166 entries, 4 to 99169
Columns: 373 entries, METER_DATE to SUM_OF_INST
dtypes: float64(372), object(1)
memory usage: 283.0+ MB
None


,METER_DATE,temp_dew_point,temp_ground,humidity,rainfall,snowfall,snowfall_3hr,wind_speed,wind_direction,pressure_vapor,...,ELEC_INST_EFF_563-22-5,ELEC_INST_EFF_563-22-6,ELEC_INST_EFF_563-23-1,ELEC_INST_EFF_563-23-2,ELEC_INST_EFF_563-23-3,ELEC_INST_EFF_563-23-4,ELEC_INST_EFF_563-24-1,ELEC_INST_EFF_563-24-2,MEAN_OF_INST,SUM_OF_INST
4,2022-07-18 00:00:00,21.3000000000,23.4000000000,85.0000000000,0.0000000000,0.0000000000,0.0000000000,0.5000000000,250.0000000000,25.3000000000,...,0.0400000000,0.0400000000,0.0400000000,0.0600000000,0.0400000000,0.0500000000,0.0300000000,0.0800000000,0.0738053097,25.0200000000
5,2022-07-18 00:10:00,21.3666666667,23.3333333333,85.6666666667,0.0000000000,0.0000000000,0.0000000000,0.5000000000,226.6666666667,25.4166666667,...,0.0400000000,0.0400000000,0.0300000000,0.0600000000,0.0600000000,0.0300000000,0.0200000000,0.0600000000,0.0706528190,23.8100000000
6,2022-07-18 00:20:00,21.4333333333,23.2666666667,86.3333333333,0.0000000000,0.0000000000,0.0000000000,0.5000000000,203.3333333333,25.5333333333,...,0.0400000000,0.0400000000,0.0300000000,0.0700000000,0.0400000000,0.0200000000,0.0300000000,0.0800000000,0.0687500000,23.6500000000
7,2022-07-18 00:30:00,21.5000000000,23.2000000000,87.0000000000,0.0000000000,0.0000000000,0.0000000000,0.5000000000,180.0000000000,25.6500000000,...,0.0200000000,0.0300000000,0.0400000000,0.0400000000,0.0300000000,0.0400000000,0.0200000000,0.0600000000,0.0646376812,22.3000000000
8,2022-07-18 00:40:00,21.5666666667,23.1333333333,87.6666666667,0.0000000000,0.0000000000,0.0000000000,0.5000000000,156.6666666667,25.7666666667,...,0.0200000000,0.0400000000,0.0400000000,0.0600000000,0.0500000000,0.0300000000,0.0400000000,0.0700000000,0.0615406977,21.1700000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99165,2024-06-05 14:50:00,14.3500000000,41.5000000000,47.0000000000,0.0000000000,0.0000000000,0.0000000000,3.0000000000,278.3333333333,16.3500000000,...,0.0200000000,0.0503743316,0.0300000000,0.0503743316,0.0503743316,0.0503743316,0.0503743316,0.0503743316,0.0503743316,9.4200000000
99166,2024-06-05 15:00:00,14.4000000000,41.1000000000,48.0000000000,0.0000000000,0.0000000000,0.0000000000,3.1000000000,270.0000000000,16.4000000000,...,0.0400000000,0.0511049724,0.0511049724,0.0511049724,0.0511049724,0.0511049724,0.0511049724,0.0511049724,0.0511049724,9.2500000000
99167,2024-06-05 15:10:00,14.4000000000,41.1000000000,48.0000000000,0.0000000000,0.0000000000,0.0000000000,3.1000000000,270.0000000000,16.4000000000,...,0.0800000000,0.0477714286,0.0477714286,0.0477714286,0.0477714286,0.0477714286,0.0477714286,0.0477714286,0.0477714286,8.3600000000
99168,2024-06-05 15:20:00,14.4000000000,41.1000000000,48.0000000000,0.0000000000,0.0000000000,0.0000000000,3.1000000000,270.0000000000,16.4000000000,...,0.0500000000,0.0453977273,0.0453977273,0.0453977273,0.0453977273,0.0453977273,0.0453977273,0.0453977273,0.0453977273,7.9900000000


In [32]:
str_file = 'KIER_' + str_domain + '_INST_Weather_C0_' + str_interval + '.csv'
df_kier_h_c0.to_csv(str_dirName_h + str_file)
print(df_kier_h_c0.info())
df_kier_h_c0

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99166 entries, 4 to 99169
Columns: 373 entries, METER_DATE to SUM_OF_INST
dtypes: float64(372), object(1)
memory usage: 283.0+ MB
None


,METER_DATE,temp_dew_point,temp_ground,humidity,rainfall,snowfall,snowfall_3hr,wind_speed,wind_direction,pressure_vapor,...,ELEC_INST_EFF_563-22-5,ELEC_INST_EFF_563-22-6,ELEC_INST_EFF_563-23-1,ELEC_INST_EFF_563-23-2,ELEC_INST_EFF_563-23-3,ELEC_INST_EFF_563-23-4,ELEC_INST_EFF_563-24-1,ELEC_INST_EFF_563-24-2,MEAN_OF_INST,SUM_OF_INST
4,2022-07-18 00:00:00,21.3000000000,23.4000000000,85.0000000000,0.0000000000,0.0000000000,0.0000000000,0.5000000000,250.0000000000,25.3000000000,...,0.0400000000,0.0400000000,0.0400000000,0.0600000000,0.0400000000,0.0500000000,0.0300000000,0.0800000000,0.0738053097,25.0200000000
5,2022-07-18 00:10:00,21.3666666667,23.3333333333,85.6666666667,0.0000000000,0.0000000000,0.0000000000,0.5000000000,226.6666666667,25.4166666667,...,0.0400000000,0.0400000000,0.0300000000,0.0600000000,0.0600000000,0.0300000000,0.0200000000,0.0600000000,0.0706528190,23.8100000000
6,2022-07-18 00:20:00,21.4333333333,23.2666666667,86.3333333333,0.0000000000,0.0000000000,0.0000000000,0.5000000000,203.3333333333,25.5333333333,...,0.0400000000,0.0400000000,0.0300000000,0.0700000000,0.0400000000,0.0200000000,0.0300000000,0.0800000000,0.0687500000,23.6500000000
7,2022-07-18 00:30:00,21.5000000000,23.2000000000,87.0000000000,0.0000000000,0.0000000000,0.0000000000,0.5000000000,180.0000000000,25.6500000000,...,0.0200000000,0.0300000000,0.0400000000,0.0400000000,0.0300000000,0.0400000000,0.0200000000,0.0600000000,0.0646376812,22.3000000000
8,2022-07-18 00:40:00,21.5666666667,23.1333333333,87.6666666667,0.0000000000,0.0000000000,0.0000000000,0.5000000000,156.6666666667,25.7666666667,...,0.0200000000,0.0400000000,0.0400000000,0.0600000000,0.0500000000,0.0300000000,0.0400000000,0.0700000000,0.0615406977,21.1700000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99165,2024-06-05 14:50:00,14.3500000000,41.5000000000,47.0000000000,0.0000000000,0.0000000000,0.0000000000,3.0000000000,278.3333333333,16.3500000000,...,0.0200000000,0.0503743316,0.0300000000,0.0503743316,0.0503743316,0.0503743316,0.0503743316,0.0503743316,0.0503743316,9.4200000000
99166,2024-06-05 15:00:00,14.4000000000,41.1000000000,48.0000000000,0.0000000000,0.0000000000,0.0000000000,3.1000000000,270.0000000000,16.4000000000,...,0.0400000000,0.0511049724,0.0511049724,0.0511049724,0.0511049724,0.0511049724,0.0511049724,0.0511049724,0.0511049724,9.2500000000
99167,2024-06-05 15:10:00,14.4000000000,41.1000000000,48.0000000000,0.0000000000,0.0000000000,0.0000000000,3.1000000000,270.0000000000,16.4000000000,...,0.0800000000,0.0477714286,0.0477714286,0.0477714286,0.0477714286,0.0477714286,0.0477714286,0.0477714286,0.0477714286,8.3600000000
99168,2024-06-05 15:20:00,14.4000000000,41.1000000000,48.0000000000,0.0000000000,0.0000000000,0.0000000000,3.1000000000,270.0000000000,16.4000000000,...,0.0500000000,0.0453977273,0.0453977273,0.0453977273,0.0453977273,0.0453977273,0.0453977273,0.0453977273,0.0453977273,7.9900000000


In [33]:
str_file = 'KIER_' + str_domain + '_INST_Weather_C1_' + str_interval + '.csv'
df_kier_h_c1.to_csv(str_dirName_h + str_file)
print(df_kier_h_c1.info())
df_kier_h_c1

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99166 entries, 4 to 99169
Columns: 373 entries, METER_DATE to SUM_OF_INST
dtypes: float64(372), object(1)
memory usage: 283.0+ MB
None


,METER_DATE,temp_dew_point,temp_ground,humidity,rainfall,snowfall,snowfall_3hr,wind_speed,wind_direction,pressure_vapor,...,ELEC_INST_EFF_563-22-5,ELEC_INST_EFF_563-22-6,ELEC_INST_EFF_563-23-1,ELEC_INST_EFF_563-23-2,ELEC_INST_EFF_563-23-3,ELEC_INST_EFF_563-23-4,ELEC_INST_EFF_563-24-1,ELEC_INST_EFF_563-24-2,MEAN_OF_INST,SUM_OF_INST
4,2022-07-18 00:00:00,21.3000000000,23.4000000000,85.0000000000,0.0000000000,0.0000000000,0.0000000000,0.5000000000,250.0000000000,25.3000000000,...,0.0400000000,0.0400000000,0.0400000000,0.0600000000,0.0400000000,0.0500000000,0.0300000000,0.0800000000,0.0738053097,25.0200000000
5,2022-07-18 00:10:00,21.3666666667,23.3333333333,85.6666666667,0.0000000000,0.0000000000,0.0000000000,0.5000000000,226.6666666667,25.4166666667,...,0.0400000000,0.0400000000,0.0300000000,0.0600000000,0.0600000000,0.0300000000,0.0200000000,0.0600000000,0.0706528190,23.8100000000
6,2022-07-18 00:20:00,21.4333333333,23.2666666667,86.3333333333,0.0000000000,0.0000000000,0.0000000000,0.5000000000,203.3333333333,25.5333333333,...,0.0400000000,0.0400000000,0.0300000000,0.0700000000,0.0400000000,0.0200000000,0.0300000000,0.0800000000,0.0687500000,23.6500000000
7,2022-07-18 00:30:00,21.5000000000,23.2000000000,87.0000000000,0.0000000000,0.0000000000,0.0000000000,0.5000000000,180.0000000000,25.6500000000,...,0.0200000000,0.0300000000,0.0400000000,0.0400000000,0.0300000000,0.0400000000,0.0200000000,0.0600000000,0.0646376812,22.3000000000
8,2022-07-18 00:40:00,21.5666666667,23.1333333333,87.6666666667,0.0000000000,0.0000000000,0.0000000000,0.5000000000,156.6666666667,25.7666666667,...,0.0200000000,0.0400000000,0.0400000000,0.0600000000,0.0500000000,0.0300000000,0.0400000000,0.0700000000,0.0615406977,21.1700000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99165,2024-06-05 14:50:00,14.3500000000,41.5000000000,47.0000000000,0.0000000000,0.0000000000,0.0000000000,3.0000000000,278.3333333333,16.3500000000,...,0.0200000000,0.0503743316,0.0300000000,0.0503743316,0.0503743316,0.0503743316,0.0503743316,0.0503743316,0.0503743316,9.4200000000
99166,2024-06-05 15:00:00,14.4000000000,41.1000000000,48.0000000000,0.0000000000,0.0000000000,0.0000000000,3.1000000000,270.0000000000,16.4000000000,...,0.0400000000,0.0511049724,0.0511049724,0.0511049724,0.0511049724,0.0511049724,0.0511049724,0.0511049724,0.0511049724,9.2500000000
99167,2024-06-05 15:10:00,14.4000000000,41.1000000000,48.0000000000,0.0000000000,0.0000000000,0.0000000000,3.1000000000,270.0000000000,16.4000000000,...,0.0800000000,0.0477714286,0.0477714286,0.0477714286,0.0477714286,0.0477714286,0.0477714286,0.0477714286,0.0477714286,8.3600000000
99168,2024-06-05 15:20:00,14.4000000000,41.1000000000,48.0000000000,0.0000000000,0.0000000000,0.0000000000,3.1000000000,270.0000000000,16.4000000000,...,0.0500000000,0.0453977273,0.0453977273,0.0453977273,0.0453977273,0.0453977273,0.0453977273,0.0453977273,0.0453977273,7.9900000000


In [34]:
str_file = 'KIER_' + str_domain + '_INST_Weather_C2_' + str_interval + '.csv'
df_kier_h_c2.to_csv(str_dirName_h + str_file)
print(df_kier_h_c2.info())
df_kier_h_c2

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99166 entries, 4 to 99169
Columns: 373 entries, METER_DATE to SUM_OF_INST
dtypes: float64(371), object(2)
memory usage: 283.0+ MB
None


,METER_DATE,temp_dew_point,temp_ground,humidity,rainfall,snowfall,snowfall_3hr,wind_speed,wind_direction,pressure_vapor,...,ELEC_INST_EFF_563-22-5,ELEC_INST_EFF_563-22-6,ELEC_INST_EFF_563-23-1,ELEC_INST_EFF_563-23-2,ELEC_INST_EFF_563-23-3,ELEC_INST_EFF_563-23-4,ELEC_INST_EFF_563-24-1,ELEC_INST_EFF_563-24-2,MEAN_OF_INST,SUM_OF_INST
4,2022-07-18 00:00:00,21.3000000000,23.4000000000,85.0000000000,0.0000000000,0.0000000000,0.0000000000,0.5000000000,250.0000000000,25.3000000000,...,0.0400000000,0.0400000000,0.0400000000,0.0600000000,0.0400000000,0.0500000000,0.0300000000,0.0800000000,0.0738053097,25.0200000000
5,2022-07-18 00:10:00,21.3666666667,23.3333333333,85.6666666667,0.0000000000,0.0000000000,0.0000000000,0.5000000000,226.6666666667,25.4166666667,...,0.0400000000,0.0400000000,0.0300000000,0.0600000000,0.0600000000,0.0300000000,0.0200000000,0.0600000000,0.0706528190,23.8100000000
6,2022-07-18 00:20:00,21.4333333333,23.2666666667,86.3333333333,0.0000000000,0.0000000000,0.0000000000,0.5000000000,203.3333333333,25.5333333333,...,0.0400000000,0.0400000000,0.0300000000,0.0700000000,0.0400000000,0.0200000000,0.0300000000,0.0800000000,0.0687500000,23.6500000000
7,2022-07-18 00:30:00,21.5000000000,23.2000000000,87.0000000000,0.0000000000,0.0000000000,0.0000000000,0.5000000000,180.0000000000,25.6500000000,...,0.0200000000,0.0300000000,0.0400000000,0.0400000000,0.0300000000,0.0400000000,0.0200000000,0.0600000000,0.0646376812,22.3000000000
8,2022-07-18 00:40:00,21.5666666667,23.1333333333,87.6666666667,0.0000000000,0.0000000000,0.0000000000,0.5000000000,156.6666666667,25.7666666667,...,0.0200000000,0.0400000000,0.0400000000,0.0600000000,0.0500000000,0.0300000000,0.0400000000,0.0700000000,0.0615406977,21.1700000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99165,2024-06-05 14:50:00,14.3500000000,41.5000000000,47.0000000000,0.0000000000,0.0000000000,0.0000000000,3.0000000000,278.3333333333,16.3500000000,...,0.0200000000,0.0503743316,0.0300000000,0.0503743316,0.0503743316,0.0503743316,0.0503743316,0.0503743316,0.0503743316,9.4200000000
99166,2024-06-05 15:00:00,14.4000000000,41.1000000000,48.0000000000,0.0000000000,0.0000000000,0.0000000000,3.1000000000,270.0000000000,16.4000000000,...,0.0400000000,0.0511049724,0.0511049724,0.0511049724,0.0511049724,0.0511049724,0.0511049724,0.0511049724,0.0511049724,9.2500000000
99167,2024-06-05 15:10:00,14.4000000000,41.1000000000,48.0000000000,0.0000000000,0.0000000000,0.0000000000,3.1000000000,270.0000000000,16.4000000000,...,0.0800000000,0.0477714286,0.0477714286,0.0477714286,0.0477714286,0.0477714286,0.0477714286,0.0477714286,0.0477714286,8.3600000000
99168,2024-06-05 15:20:00,14.4000000000,41.1000000000,48.0000000000,0.0000000000,0.0000000000,0.0000000000,3.1000000000,270.0000000000,16.4000000000,...,0.0500000000,0.0453977273,0.0453977273,0.0453977273,0.0453977273,0.0453977273,0.0453977273,0.0453977273,0.0453977273,7.9900000000
